# LimberJack Target Practice

## Model

In [1]:
using LinearAlgebra
using Turing
using AdvancedHMC
using LimberJack
using DataFrames
using CSV
using YAML
using NPZ
using PythonCall
sacc = pyimport("sacc");

using Revise
using MCHMC

LoadError: expected a single entry for "LaTeXStrings" in "/home/jaimerz/PhD/MCHMC.jl/Project.toml"

In [2]:
sacc_path = "/home/jaimerz/PhD/LimberJack.jl/data/FD/cls_FD_covG.fits"
yaml_path = "/home/jaimerz/PhD/LimberJack.jl/data/DESY1/DESY1.yml"
sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)
meta, files = make_data(sacc_file, yaml_file)

data_vector = meta.data
cov_tot = meta.cov
errs = sqrt.(diag(cov_tot))
fake_data = data_vector ./ errs
fake_cov = Hermitian(cov_tot ./ (errs * errs'));

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13
DESgc__0 DESwl__0 5
DESgc__0 DESwl__1 5
DESgc__0 DESwl__2 5
DESgc__0 DESwl__3 5
DESgc__1 DESwl__0 8
DESgc__1 DESwl__1 8
DESgc__1 DESwl__2 8
DESgc__1 DESwl__3 8
DESgc__2 DESwl__0 10
DESgc__2 DESwl__1 10
DESgc__2 DESwl__2 10
DESgc__2 DESwl__3 10
DESgc__3 DESwl__0 11
DESgc__3 DESwl__1 11
DESgc__3 DESwl__2 11
DESgc__3 DESwl__3 11
DESgc__4 DESwl__0 13
DESgc__4 DESwl__1 13
DESgc__4 DESwl__2 13
DESgc__4 DESwl__3 13
DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


In [3]:
@model function model(data;
                      cov=fake_cov,
                      meta=meta, 
                      files=files)
    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ TruncatedNormal(72, 5, 0.64, 0.82)
    s8 ~ Uniform(0.4, 1.2)
    ns ~ Uniform(0.84, 1.1)

    DESgc__0_b ~ Uniform(0.8, 3.0)
    DESgc__1_b ~ Uniform(0.8, 3.0)
    DESgc__2_b ~ Uniform(0.8, 3.0)
    DESgc__3_b ~ Uniform(0.8, 3.0)
    DESgc__4_b ~ Uniform(0.8, 3.0)
    DESgc__0_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__1_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__2_dz ~ TruncatedNormal(0.0, 0.006, -0.2, 0.2)
    DESgc__3_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESgc__4_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESwl__0_dz ~ TruncatedNormal(-0.001, 0.016, -0.2, 0.2)
    DESwl__1_dz ~ TruncatedNormal(-0.019, 0.013, -0.2, 0.2)
    DESwl__2_dz ~ TruncatedNormal(0.009, 0.011, -0.2, 0.2)
    DESwl__3_dz ~ TruncatedNormal(-0.018, 0.022, -0.2, 0.2)
    DESwl__0_m ~ Normal(0.012, 0.023)
    DESwl__1_m ~ Normal(0.012, 0.023)
    DESwl__2_m ~ Normal(0.012, 0.023)
    DESwl__3_m ~ Normal(0.012, 0.023)
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("DESgc__0_b" => DESgc__0_b,
                     "DESgc__1_b" => DESgc__1_b,
                     "DESgc__2_b" => DESgc__2_b,
                     "DESgc__3_b" => DESgc__3_b,
                     "DESgc__4_b" => DESgc__4_b,
                     "DESgc__0_dz" => DESgc__0_dz,
                     "DESgc__1_dz" => DESgc__1_dz,
                     "DESgc__2_dz" => DESgc__2_dz,
                     "DESgc__3_dz" => DESgc__3_dz,
                     "DESgc__4_dz" => DESgc__4_dz,
                     "DESwl__0_dz" => DESwl__0_dz,
                     "DESwl__1_dz" => DESwl__1_dz,
                     "DESwl__2_dz" => DESwl__2_dz,
                     "DESwl__3_dz" => DESwl__3_dz,
                     "DESwl__0_m" => DESwl__0_m,
                     "DESwl__1_m" => DESwl__1_m,
                     "DESwl__2_m" => DESwl__2_m,
                     "DESwl__3_m" => DESwl__3_m,
                     "A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,)

    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit")

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    data ~ MvNormal(theory ./ errs, cov)
end

model (generic function with 2 methods)

In [4]:
stat_model = model(fake_data)

DynamicPPL.Model{typeof(model), (:data, :cov, :meta, :files), (:cov, :meta, :files), (), Tuple{Vector{Float64}, Hermitian{Float64, Matrix{Float64}}, LimberJack.Meta, Dict{String, Vector}}, Tuple{Hermitian{Float64, Matrix{Float64}}, LimberJack.Meta, Dict{String, Vector}}, DynamicPPL.DefaultContext}(model, (data = [3.214096950162085, 4.70756721223778, 6.415689628953242, 5.828388701788672, 8.493679322382144, 2.9304058751712, 7.998464834088993, 5.1210836387672565, 8.932453735970117, 8.407292870179436  …  1.8338144344899103, 3.6747343455612156, 2.7292480563019765, 2.265947203686935, 3.1513497907284673, 2.887098863974196, 0.7659895057386662, 2.8855449419318098, 2.3446896354956963, 1.6352516474328562], cov = [1.0 -0.289532718133784 … -4.320136350638793e-5 -4.4205040697944595e-5; -0.289532718133784 0.9999999999999998 … -9.461726942265441e-5 -9.845166181745778e-5; … ; -4.320136350638793e-5 -9.461726942265441e-5 … 0.9999999999999998 0.0033632257724296594; -4.4205040697944595e-5 -9.84516618174577

## Sampling

In [5]:
target = TuringTarget(stat_model);

LoadError: UndefVarError: `TuringTarget` not defined

In [6]:
eps=0.5
L=sqrt(20)
mchmc = Sampler(eps, L, integrator="MN")

LoadError: UndefVarError: `Sampler` not defined

In [7]:
samples_mchmc = Sample(mchmc, target, 1; monitor_energy=true, dialog=true)

LoadError: UndefVarError: `Sample` not defined

In [8]:
using AverageShiftedHistograms, Plots

In [9]:
Wms_mchmc = [sample[1] for sample in samples_mchmc]
s8s_mchmc = [sample[4] for sample in samples_mchmc];

LoadError: UndefVarError: `samples_mchmc` not defined

In [10]:
Wm_mchmc_ker = ash(Wms_mchmc; m=30)
s8_mchmc_ker = ash(s8s_mchmc; m=30)

LoadError: UndefVarError: `Wms_mchmc` not defined

In [11]:
Plots.plot(Wm_mchmc_ker; hist=false)

LoadError: UndefVarError: `Wm_mchmc_ker` not defined

In [12]:
Plots.plot(s8_mchmc_ker; hist=false)

LoadError: UndefVarError: `s8_mchmc_ker` not defined

## NUTS

In [13]:
using Pkg
Pkg.add("DataFrames")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/PhD/MCHMC.jl/Project.toml`
  No Changes to `~/PhD/MCHMC.jl/Manifest.toml`


In [14]:
#samples_hmc = sample(stat_model, NUTS(500, 0.65),
#                  500, progress=true; save_state=true)

summary_hmc = CSV.read("/home/jaimerz/PhD/LimberJack.jl/chains/DESY1_k1k_priors_EisHu_TAP_0.65/summary_1.csv", DataFrame)
samples_hmc = CSV.read("/home/jaimerz/PhD/LimberJack.jl/chains/DESY1_k1k_priors_EisHu_TAP_0.65/chain_1.csv", DataFrame);

In [15]:
summary_hmc.ess

25-element Vector{Float64}:
  107.13649180104869
  331.4835789816811
  207.484061400768
  111.64048654758332
  600.4013034569028
  114.39496059843647
  117.33719647252566
  114.56966849849671
  111.15547545924896
  105.06968640469209
  961.3377488207435
  700.0507524844818
  837.2166448905194
  682.3676139298411
  566.0490257440192
  837.5008331222044
  733.9973963424983
  549.1048795030388
  792.5424251430129
 1009.6818545217956
  763.2299996132994
  872.0250289709837
  763.7811879280131
  552.5319549420313
  731.942598764305

In [16]:
Wms_hmc = vec(samples_hmc[!, "Ωm"])
s8s_hmc = vec(samples_hmc[!, "s8"]);

In [17]:
Wm_hmc_ker = ash(Wms_hmc; m=30)
s8_hmc_ker = ash(s8s_hmc; m=30)
Wm_mchmc_ker = ash(Wms_mchmc; m=30)
s8_mchmc_ker = ash(s8s_mchmc; m=30);

LoadError: UndefVarError: `Wms_mchmc` not defined

In [18]:
Plots.plot(Wm_mchmc_ker; hist=false, label="MCHMC")
Plots.plot!(Wm_hmc_ker; hist=false, label="HMC", xlabel="Wm")

LoadError: UndefVarError: `Wm_mchmc_ker` not defined

In [19]:
Plots.plot(s8_mchmc_ker; hist=false, label="MCHMC")
Plots.plot!(s8_hmc_ker; hist=false, label="HMC",  xlabel="s8")

LoadError: UndefVarError: `s8_mchmc_ker` not defined

## Self-tuning

In [20]:
mchmc = Sampler()
println(string("L = ", mchmc.settings.L, ", eps = ", mchmc.settings.eps))
# This should start self-tuning    

LoadError: UndefVarError: `Sampler` not defined

In [21]:
mchmc.settings.tune_samples

LoadError: UndefVarError: `mchmc` not defined

In [22]:
samples = Sample(mchmc, target; num_steps=10000, dialog=true);

LoadError: UndefVarError: `Sample` not defined